In [13]:
import requests
from fastapi import FastAPI,Header,Body
from pydantic import BaseModel
from nest_asyncio import apply
import uvicorn
from typing import Annotated

from typing import Any
from fastapi.responses import RedirectResponse
from fastapi.responses import ORJSONResponse

from fastapi.encoders import jsonable_encoder
from fastapi.responses import HTMLResponse,PlainTextResponse
from fastapi.responses import Response,PlainTextResponse
from fastapi.responses import StreamingResponse
from fastapi.encoders import jsonable_encoder
from fastapi.responses import HTMLResponse,PlainTextResponse
from fastapi.responses import Response,PlainTextResponse
from fastapi.responses import StreamingResponse
from pydantic import EmailStr
import json
import hashlib


In [10]:
class User(BaseModel):
    user_name : str
    user_email : EmailStr

In [18]:
app = FastAPI()

fake_db = {}

class User(BaseModel):
    username: str
    fullname: str
    email: EmailStr
    password: str

class User_Login(BaseModel):
    username: str
    password: str
    
    
def hash_password(password: str) -> str:
    hash_object = hashlib.sha256(password.encode())
    return hash_object.hexdigest()

@app.post('/create_user/')
async def create_user(data: User):
    if data.username in fake_db:
        return {"message": f"User {data.username} already exists."}
    
    hashed_password = hash_password(data.password)
    user_info = data.dict()
    user_info['password'] = hashed_password
    fake_db[data.username] = user_info
    
    return {"message": "User created successfully."}

@app.post('/login/')
async def login(data: User_Login):
    if data.username not in fake_db:
        return {"message": f"User {data.username} are not exists. plz create a account"}
    
    stored_password = fake_db[data.username].get('password')
    if stored_password != hash_password(data.password):
        return {"message": f"incorect Password"}
    
    return {"message": f"Welcome {fake_db[data.username]['fullname']}"}

@app.get("/users/")
async def users():
    tmp = fake_db.copy()
    for user_info in tmp.values():
        user_info.pop("password", None)
    return tmp

if __name__ == "__main__":
    apply()
    uvicorn.run(app, port=4444)

INFO:     Started server process [1187240]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:48122 - "POST /create_user/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:55360 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:46730 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58428 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:59062 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:59066 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:35090 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:35106 - "POST /login/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:49228 - "POST /create_user/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:34206 - "GET /users/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:47876 - "GET /users/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:47888 - "GET /users/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52130 - "GET /users/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:34414 - "GET /users/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1187240]
